
# Implementing efficient recorders

When taking time series data during a simulation, a callable gets passed in and called every generation.  Therefore, the details of that callable can have a big effect on run time.  Here, we explore different ways to write such "recorders" and benchmark them.

The first thing we need is a function that we can reuse in order to evolve a population with a recorder.  That function is shown in the next block.  We are opting to require that recorders be constructed with the length of the simulation (in generation).  This requirement means that our recorders can pre-allocate the memory for recorded data, which is an optional optimization.

Here's the code for evolving a function:

In [2]:
import fwdpy11
import fwdpy11.model_params
import fwdpy11.fitness
import fwdpy11.wright_fisher
import numpy as np
import pandas as pd

def evolve(args):
    """
    Evolve function takes an initial
    population size,
    RNG seed, and the type name of a recorder as
    arguments.
    """
    N,seed,recorderType,theta,rho,simlen=args
    pop = fwdpy11.SlocusPop(N)
    rng=fwdpy11.GSLrng(seed)
    nrate=theta/float(4*N)
    recrate=rho/float(4*N)
    params=fwdpy11.model_params.SlocusParams(
        nregions=[fwdpy11.Region(0,1,1)],
        sregions=[fwdpy11.ExpS(0,1,1,-0.1,1.0)],
        recregions=[fwdpy11.Region(0,1,1)],
        gvalue=fwdpy11.fitness.SlocusAdditive(2.0),
        demography=np.array([N]*simlen,dtype=np.uint32),
        rates=(nrate,5e-3,recrate))
    recorder = None
    if recorderType is not None:
        try:
            recorder=recorderType(simlen,N)
        except:
            recorder=recorderType(simlen)
    print(type(recorder))
    fwdpy11.wright_fisher.evolve(rng,pop,params,recorder)
    if recorder is not None:
        return recorder.data

First things first--how long does a simulation take with no recorder?

In [3]:
%timeit evolve((1000,42,None,100.,100.,500))

<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
1 loop, best of 3: 218 ms per loop


## Implementing recorders in Python

Our first recorder will do the following:

1. Calculate population mean fitness
2. Calculate the sum of effect sizes on each gamete in each diploid, and store the mean.

This is not the most interesting thing to record about a population, but it is a useful example because it requires operations on most of the data in a population.

Here is our first version, written in fairly plain Python.  Our only NumPy use is for intermediate data containers:

In [4]:
class PyRecorder(object):
    def __init__(self,simlen,N):
        #An index variable referring to where we will next store data
        self.i = 0
        #Pre-allocate a structured array for our data.
        #s1 and s2 are the mean sums of effect sizes on gamete 1 and 2, resp.
        self.data=np.array([(-1,np.nan,np.nan,np.nan)]*simlen,
                          dtype=[('generation',np.uint32),
                                ('wbar',np.float),
                                ('s1',np.float),
                                ('s2',np.float)])
        self.fitness = np.zeros(N,dtype=np.float)
        self.nmuts1 = np.zeros(N,dtype=np.float)
        self.nmuts2 = np.zeros(N,dtype=np.float)
    def __call__(self,pop):
        j=0
        for dip in pop.diploids:
            self.fitness[j]=dip.w
            n1 = 0.
            for key in pop.gametes[dip.first].smutations:
                n1 += pop.mutations[key].s
            n2 = 0.
            for key in pop.gametes[dip.second].smutations:
                n2 += pop.mutations[key].s
            self.nmuts1[j]=n1
            self.nmuts2[j]=n2
            j+=1
        self.data[self.i] = (pop.generation,
                             self.fitness.mean(),
                             self.nmuts1.mean(),
                             self.nmuts2.mean())
        self.i += 1

In [5]:
%timeit x=evolve((1000,42,PyRecorder,100.,100.,500))

<class '__main__.PyRecorder'>
<class '__main__.PyRecorder'>
<class '__main__.PyRecorder'>
<class '__main__.PyRecorder'>
1 loop, best of 3: 54.7 s per loop


In [6]:
class PyRecorder2(object):
    def __init__(self,simlen):
        self.i = 0
        self.data=np.array([(-1,np.nan,np.nan,np.nan)]*simlen,
                          dtype=[('generation',np.uint32),
                                ('wbar',np.float),
                                ('s1',np.float),
                                ('s2',np.float)])
    def __call__(self,pop):
        wbar = np.array(pop.diploids.trait_array())['w'].mean()
        esizes = np.array(pop.mutations.array())['s']
        j=0
        nm1=0.
        nm2=0.
        for dip in pop.diploids:
            nm1 += esizes[np.array(pop.gametes[dip.first].smutations)].sum()
            nm2 += esizes[np.array(pop.gametes[dip.second].smutations)].sum()
            j+=1
            
        self.data[self.i] = (pop.generation,wbar,nm1/float(pop.N),nm2/float(pop.N))
        self.i += 1

In [7]:
%timeit x=evolve((1000,42,PyRecorder2,100.,100.,500))

<class '__main__.PyRecorder2'>
<class '__main__.PyRecorder2'>
<class '__main__.PyRecorder2'>
<class '__main__.PyRecorder2'>
1 loop, best of 3: 36.6 s per loop


## An aside

It is important to keep in mind that not all Python-based samplers need affect run times to the degree shown above!  The above examples are somewhat designed to be worst-case, as they are forcing Python to do a lot of stuff that it cannot optimize.  Let's take a look at another sampler, which generates a genotype matrix for 200 diploids: 

In [8]:
import fwdpy11.sampling

np.random.seed(42)
class MatrixSampler(object):
    def __init__(self,simlen):  #the simlen arg is ignored
        self.data=[]
    def __call__(self,pop):
        individuals = np.random.choice(pop.N,200,replace=False)
        keys = fwdpy11.sampling.mutation_keys(pop,individuals)
        neutral_sorted_keys=[i for i in sorted(keys[0],key=lambda x,m=pop.mutations: m[x[0]].pos)]
        selected_sorted_keys=[i for i in sorted(keys[1],key=lambda x,m=pop.mutations: m[x[0]].pos)]
        dm = fwdpy11.sampling.genotype_matrix(pop,individuals,neutral_sorted_keys,selected_sorted_keys)
        self.data.append(dm)

In [9]:
%timeit x=evolve((1000,42,MatrixSampler,100.,100.,500))

<class '__main__.MatrixSampler'>
<class '__main__.MatrixSampler'>
<class '__main__.MatrixSampler'>
<class '__main__.MatrixSampler'>
1 loop, best of 3: 2 s per loop


Not bad!  The reason is because most of fwdpy11's built-in functionality is using C++ behind the scenes.

## Using Cython to speed things up.

So our Python-only recorders are slowing things down a lot.  Let's see if we can improve things with Cython.  The key here is to use Cython's typed memory views and "directives" to move as many operations as possible from Python to C++.  We will also leverage fwpy11's integration with Numpy to get extra speedups.

In [10]:
%load_ext Cython

In [11]:
%%cython --cplus --compile-args=-O3 --compile-args=-std=c++11 -3
from libcpp.vector cimport vector
from libc.stdint cimport uint32_t
from cython.view cimport array as cvarray
import numpy as np
cimport numpy as np
cimport cython

#A nice feature of Cython is
#implicit struct-to-dict conversion,
#so we define a structure to hold our data:
cdef struct my_data:
    uint32_t generation
    double wbar,s1,s2

@cython.boundscheck(False)
@cython.wraparound(False)
cdef inline double sum_e(uint32_t[:] keys,double[:] esizes) nogil:
    cdef size_t i = 0, l = keys.shape[0]
    cdef double t = 0.
    while i < l:
        t += esizes[keys[i]]
        i+=1
    return t

cdef class CyRecorder(object):
    #Cython can convert C++ vectors to
    #Python lists.  Thus, we can hold
    #our struct in a vector and it'll 
    #be converted to a list of dicts
    #when we ask for it in Python.
    #We mark the data as a readonly
    #attribute so that users can't modify it.
    cdef readonly vector[my_data] data
    cdef double[:] esizes_view
    def __cinit__(self,uint32_t simlen):
        self.data = vector[my_data]()
        self.data.reserve(simlen)
    @cython.cdivision(True)
    @cython.boundscheck(False)
    @cython.wraparound(False)
    def __call__(self,pop):
        cdef double wbar = np.array(pop.diploids.trait_array())['w'].mean()
        cdef np.ndarray[double,ndim=1] esizes = np.array(pop.mutations.array())['s']
        cdef my_data d
        self.esizes_view = esizes
        cdef double N = <double>(pop.N)
        cdef double t1=0.,t2=0.
        for dip in pop.diploids:
            t1 += sum_e(pop.gametes[dip.first].smutations,self.esizes_view)
            t2 += sum_e(pop.gametes[dip.second].smutations,self.esizes_view)
        d.generation = pop.generation
        d.wbar = wbar
        d.s1 = t1/N
        d.s2 = t2/N
        self.data.push_back(d)

In [12]:
%timeit x=evolve((1000,42,CyRecorder,100.,100.,500))

<class '_cython_magic_c1fa566bdd971718f341148a046f55b5.CyRecorder'>
<class '_cython_magic_c1fa566bdd971718f341148a046f55b5.CyRecorder'>
<class '_cython_magic_c1fa566bdd971718f341148a046f55b5.CyRecorder'>
<class '_cython_magic_c1fa566bdd971718f341148a046f55b5.CyRecorder'>
1 loop, best of 3: 27.5 s per loop


We see that Cython helps, but we're nowhere near native C/C++ performance.  The reason is that not all of the above code can be optimized to run as "pure" C++.  In particular, Cython's code annotator shows that the loop over diploids will spend a lot of time in Python operations.  The reason for this behavior is that Cython knows nothing about the fwdpp/fwdpy11 C++ types.  To improve performance in Cython, we'd have to make it aware of those types. However, that may be very difficult and/or impossible--I haven't been able to figure out how to get Cython to understand C++ types wrapped by pybind11.  In the end, that is probably OK, as the amount of code needed to wrap C++ types for Cython is more than what is needed to simply write the sampler in C++ and pybind11.

## A C++ implementation

We now turn to implementing our sampler using C++ and [pybind11](https://pybind11.readthedocs.io/).

As with our Cython recorder, we'll have a C-style structure representing our data.  We'll register that structure as a NumPy dtype.  Further, we'll register a vector of that structure as an "opaque" container compatible with Python's buffer protocol.  The end result will be that the recorder data will be held in a C++ vector that we can coerce directly to a NumPy structured array.

Our C++ code is shown below:

We'll use [cppimport](https://github.com/tbenthompson/cppimport) to compile and import our new module under the name "sampler":

In [13]:
import cppimport
cppimport.set_quiet(True)
sampler = cppimport.imp("sampler")

Let's benchmark it:

In [14]:
%timeit x=evolve((1000,42,sampler.cppRecorder,100.,100.,500))

<class 'sampler.cppRecorder'>
<class 'sampler.cppRecorder'>
<class 'sampler.cppRecorder'>
<class 'sampler.cppRecorder'>
1 loop, best of 3: 221 ms per loop


Take a look at that number. It is almost as fast as running a simulation without a sampler!  There's an important lesson here: if you want to maximize performance, move as much of the computation as possible outside of Python and into C/C++.  Of course, this simply must be the case, given how fwdpy11 is implemented.

## A sanity check

Do all of the above recorders give the same results? As usual with floating point calculations, the answer is "no" in an absolute sense, and "yes" in the sense of numerical precision.

Let's get the data from each sampler:

In [15]:
py1res = pd.DataFrame(evolve((1000,42,PyRecorder,100.,100.,500)))

<class '__main__.PyRecorder'>


In [16]:
py2res = pd.DataFrame(evolve((1000,42,PyRecorder2,100.,100.,500)))

<class '__main__.PyRecorder2'>


In [17]:
cyres = pd.DataFrame(evolve((1000,42,CyRecorder,100.,100.,500)))

<class '_cython_magic_c1fa566bdd971718f341148a046f55b5.CyRecorder'>


In [18]:
cppres = pd.DataFrame(np.array(evolve((1000,42,sampler.cppRecorder,100.,100.,500))))

<class 'sampler.cppRecorder'>


Let's compare the results.  We'll look at the max difference for each output column.

In [19]:
dfs=[py1res,py2res,cyres,cppres]
for i in range(len(dfs)-1):
    for j in range(i+1,len(dfs)):
        print((dfs[i]['wbar']-dfs[j]['wbar']).max(),
              (dfs[i]['s1']-dfs[j]['s1']).max(),
              (dfs[i]['s2']-dfs[j]['s2']).max())

0.0 1.12757025938e-17 1.04083408559e-17
0.0 1.12757025938e-17 1.04083408559e-17
4.32986979604e-15 1.12757025938e-17 8.67361737988e-18
0.0 0.0 0.0
4.32986979604e-15 9.54097911787e-18 4.33680868994e-18
4.32986979604e-15 9.54097911787e-18 4.33680868994e-18


## Having our cake and eating it, too.

The above results show that C++ recorders have the best run-time performance. It is tempting to think that this is a huge problem, implying that you cannot have the incredible flexibility of Python and the high-performance of C++ in the same recorder type.  Well, you can, thanks to [pybind11](https://pybind11.readthedocs.io/).  Let's write a C++ recorder that does the following:

1. Like our previous example, it holds the data in structures that can be easily converted to NumPy arrays.
2. The C++ object contains a Python function that is called.  This function processes our buffer in some way that only Python can do.

We'll show the use of this type in Python first, and save the C++ code for the end.  We will define a callable Python class that takes the buffer from C++, converts it to a NumPy record array, converts that record array to a Pandas DataFrame, and keeps appending to that data frame.

We're also doing to demand that our recorder do more work.  It will record the generation number, individual label, individual genetic value, and the sum of effect sizes on each gamete, for each individual in every generation.  In other words, instead of storing a few numbers per generation, we'll store more than $N$ values.

In [20]:
class ConvertToDF(object):
    def __init__(self):
        self.data = pd.DataFrame()
    def __call__(self,data):
        #if self.data is None:
        #    self.data = pd.DataFrame(np.array(data))
        #else:
        self.data = self.data.append(pd.DataFrame(np.array(data)))

In order for this work work, we need our recorder constructor to take a python callable as an argument, and so we have to rework our evolve function a bit:

In [21]:
def evolve2(args):
    """
    Evolve function takes an initial
    population size,
    RNG seed, and the type name of a recorder as
    arguments.
    """
    N,seed,recorderType,recorderOptions,theta,rho,simlen=args
    pop = fwdpy11.SlocusPop(N)
    rng=fwdpy11.GSLrng(seed)
    nrate=theta/float(4*N)
    recrate=rho/float(4*N)
    params=fwdpy11.model_params.SlocusParams(
        nregions=[fwdpy11.Region(0,1,1)],
        sregions=[fwdpy11.ExpS(0,1,1,-0.1,1.0)],
        recregions=[fwdpy11.Region(0,1,1)],
        gvalue=fwdpy11.fitness.SlocusAdditive(2.0),
        demography=np.array([N]*simlen,dtype=np.uint32),
        rates=(nrate,5e-3,recrate))
    #This is the difference from the function used above:
    ro = recorderOptions()
    recorder = recorderType(simlen,ro)
    fwdpy11.wright_fisher.evolve(rng,pop,params,recorder)
    return ro.data

Compile and import our module:

In [22]:
sampler_fxn = cppimport.imp("sampler_pyfunction")

Let's simulate a larger $N$, $\theta$, and $\rho$ than before:

In [23]:
%timeit x=evolve2((10000,42,sampler_fxn.cppRecorderFunc,ConvertToDF,1000.,1000.,500))

1 loop, best of 3: 38.2 s per loop


And here are the params we've done all along:

In [24]:
%timeit x=evolve2((1000,42,sampler_fxn.cppRecorderFunc,ConvertToDF,100.,100.,500))

1 loop, best of 3: 4.27 s per loop


That's pretty good performance, especially considering that our sampler is rather "dumb" in the sense that there are better ways to do what it is doing.  (The smarter thing would be to store all the data as a NumPy structured array and grow it using `numpy.concatenate`.  Structured arrays are faster, and require less memory, than DataFrames.  You only need a DataFrame when you need to take advantage of its nice indexing features.)

In [25]:
x=evolve2((10000,42,sampler_fxn.cppRecorderFunc,ConvertToDF,1000.,1000.,500))

In [26]:
type(x)

pandas.core.frame.DataFrame

There are indeed $N$ entries per generation:

In [27]:
xg=x.groupby(['generation'])
for n,g in list(xg)[:3]:
    print(len(g.index))

10000
10000
10000


And we can process things in the usual way:

In [28]:
for n,g in list(xg)[:3]:
    print(g['g'].mean(),g['s1'].mean(),g['s2'].mean())

0.9991993659973306 -0.0003946363268985555 -0.00040599767577111095
0.9981428395551999 -0.0009885146054695922 -0.0008686458393304458
0.9976957480778578 -0.0012369414885845457 -0.0010673104335580225


So how does this magic work?  The answer is in the C++ code.  The comments indicate the concepts that distinguish this example from the C++ example above.